In [1]:
# Iris Data

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
import sklearn.linear_model as lm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import xgboost as xgb

# Data Files

# kaggle.X1.names - primary feature names, in csv format
# kaggle.X1.train - primary feature values, training data set
# kaggle.X2.train - raw IR3 local image patches, training data set
# kaggle.Y.train - target values, training data set
# kaggle.X1.test - primary feature values, test data set
# kaggle.X2.test - raw IR3 local image patches, test data set

# Setup From 
# https://www.kaggle.com/yuansun/santander-customer-satisfaction/lb-0-84-for-starters/code

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


# load data
df_train_1 = pd.read_csv('./data/kaggle.X1.train.txt', header=None)
df_train_2 = pd.read_csv('./data/kaggle.X2.train.txt', header=None)
df_train_Y = pd.read_csv('./data/kaggle.Y.train.txt', header=None)
df_test_1 = pd.read_csv('./data/kaggle.X1.test.txt', header=None)
df_test_2 = pd.read_csv('./data/kaggle.X1.test.txt', header=None)


# # remove constant columns
# remove = []
# for col in df_train.columns:
#     if df_train[col].std() == 0:
#         remove.append(col)

# df_train.drop(remove, axis=1, inplace=True)
# df_test.drop(remove, axis=1, inplace=True)

# # remove duplicated columns
# remove = []
# c = df_train.columns
# for i in range(len(c)-1):
#     v = df_train[c[i]].values
#     for j in range(i+1,len(c)):
#         if np.array_equal(v,df_train[c[j]].values):
#             remove.append(c[j])

# df_train.drop(remove, axis=1, inplace=True)
# df_test.drop(remove, axis=1, inplace=True)

# y_train = df_train['TARGET'].values
# X_train = df_train.drop(['ID','TARGET'], axis=1).values

# id_test = df_test['ID']
# X_test = df_test.drop(['ID'], axis=1).values

# # length of dataset
# len_train = len(X_train)
# len_test  = len(X_test)

# # classifier
# clf = xgb.XGBClassifier(missing=np.nan, max_depth=5, n_estimators=350, learning_rate=0.03, nthread=4, subsample=0.95, colsample_bytree=0.85, seed=4242)

# X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

# # fitting
# clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])

# print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

# # predicting
# y_pred= clf.predict_proba(X_test)[:,1]

# submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
# submission.to_csv("submission.csv", index=False)

# print('Completed!')




In [5]:
# Combine Train Data 
df_train_X = pd.concat([df_train_1, df_train_2], axis=1)

# Combine Test Data 
df_test_X = pd.concat([df_test_1, df_test_2], axis=1)


# Create Numpy Arrays
X_train = df_train_X.values
y_train = df_train_Y.values
X_test = df_test_X.values

X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

# classifier
'''
best_avg_dist = 1
max_depth_val = 0
n_estimators_val = 0
learning_rate_val = 0
for i in range(4,8,2):
    for j in range(1,351,10):
        for k in [0.00003,0.0003,0.003,0.03,0.3]:
            print("i: {0}\tj: {1}\tk: {2}".format(i, j, k))
            clf = xgb.XGBRegressor(missing=np.nan, max_depth=i, n_estimators=j, learning_rate=k, nthread=12, subsample=0.9, colsample_bytree=0.85, seed=4242)
            clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="rmse", eval_set=[(X_eval, y_eval)])
            y_pred= clf.predict(X_test)
            avg_dist = abs(np.average(y_train)-np.average(y_pred))
            print("\tAvg Distance: {0}".format(avg_dist))
            if (avg_dist < best_avg_dist):
                best_avg_dist = avg_dist
                max_depth_val = i
                n_estimators_val = j
                learning_rate_val = k
'''                

#print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

clf = xgb.XGBRegressor(missing=np.nan, max_depth=6, n_estimators=10, learning_rate=0.003, nthread=12, subsample=0.9, colsample_bytree=0.85, seed=4242)
clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="rmse", eval_set=[(X_eval, y_eval)])

# predicting
y_pred= clf.predict(X_test)

print("Shape of prediction")
print(df_test_1.shape)

with open("prediction.csv", "wb") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["ID", "Prediction"])
    counter = 1
    for item in y_pred:
        writer.writerow([counter, item])
        counter = counter + 1

print('Completed!')

print("Best Distance: {0}\n\tmax_depth: {1}\n\tn_estimators: {2}\n\tlearning_rate: {3}".format(best_avg_dist, max_depth_val, n_estimators_val, learning_rate_val))

aaas

Will train until validation_0 error hasn't decreased in 20 rounds.
[0]	validation_0-rmse:0.844143
[1]	validation_0-rmse:0.843119
[2]	validation_0-rmse:0.842099
[3]	validation_0-rmse:0.841067
[4]	validation_0-rmse:0.840056
[5]	validation_0-rmse:0.839024
[6]	validation_0-rmse:0.838011
[7]	validation_0-rmse:0.837011
[8]	validation_0-rmse:0.835992


Shape of prediction
(40000, 91)
Completed!
Best Distance: 0.00152168312501
	max_depth: 6
	n_estimators: 101
	learning_rate: 0.003


[9]	validation_0-rmse:0.834996
